# Analyze results
Aug 28, 2020

In [41]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize


In [42]:
from ipywidgets import *

In [43]:
%matplotlib widget

In [44]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/')
from modules_image_analysis import *

In [45]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [46]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [47]:
dict1={'128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/512sq/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [48]:
# parent_dir=u.result
parent_dir=dict1['128']
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20201002_073628', '20201029_180105_no_adv_loss', '20…

In [49]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

### Read loss data
with open (result_dir+'/metrics.pickle','rb') as f:
    metrics_dict=pickle.load(f)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201029_180105_no_adv_loss


In [50]:
metrics_dict

{'Dreal': array([156.06436157, 169.06518555, 150.99403381, ...,   0.34510508,
          0.74329376,          nan]),
 'Dfake': array([9.05514453e+03, 8.55813477e+03, 8.36160742e+03, ...,
        1.10421516e-01, 9.57372263e-02,            nan]),
 'Dfull': array([9.21120898e+03, 8.72720020e+03, 8.51260156e+03, ...,
        4.55526590e-01, 8.39030981e-01,            nan]),
 'G_adv': array([0.        , 0.        , 0.        , ..., 8.48460865, 6.60236835,
               nan]),
 'G_full': array([56.378582  , 51.42838287, 48.79014587, ..., 34.65065765,
        33.14324188,         nan]),
 'spec_loss': array([56.378582  , 51.42838287, 48.79014587, ..., 34.65065765,
        33.14324188,         nan]),
 'hist_loss': array([-3.77119803,  2.4188695 ,  4.07172775, ...,  2.5966723 ,
         2.64232635,         nan]),
 'spec_chi': array([        nan, 50.82799911, 48.71123886, ..., 33.54552078,
        36.83777618, 37.31142044]),
 'hist_chi': array([       nan, 3.76698136, 4.02478647, ..., 2.63078308,

## Plot Losses

In [51]:
df_metrics=pd.DataFrame([])
for key in metrics_dict.keys(): df_metrics[key]=metrics_dict[key]
df_metrics['step']=np.arange(df_metrics.shape[0])
# df_metrics=df_metrics.dropna()

In [52]:
df_metrics.step.values.shape,df_metrics.G_full.values.shape

((31251,), (31251,))

In [53]:
df_metrics.head(10)

,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,step
0,156.064362,9055.144531,9211.208984,0.0,56.378582,56.378582,-3.771198,NaN,NaN,0
1,169.065186,8558.134766,8727.200195,0.0,51.428383,51.428383,2.418869,50.827999,3.766981,1
2,150.994034,8361.607422,8512.601562,0.0,48.790146,48.790146,4.071728,48.711239,4.024786,2
3,157.047485,8585.446289,8742.494141,0.0,48.054020,48.054020,4.026848,48.289909,3.957490,3
4,159.792755,7800.417969,7960.210938,0.0,49.900719,49.900719,3.916519,48.084919,4.023258,4
5,149.923355,8284.544922,8434.468750,0.0,48.616043,48.616043,4.053534,48.612499,4.075822,5
6,172.202179,8064.131836,8236.333984,0.0,48.534889,48.534889,2.687699,49.605465,-5.016946,6
7,172.233185,7447.735352,7619.968750,0.0,48.349136,48.349136,4.058213,48.256588,3.719382,7
8,153.595428,7685.486328,7839.081543,0.0,48.036930,48.036930,4.012977,48.150833,4.018581,8
9,165.105759,6725.641602,6890.747559,0.0,48.122658,48.122658,3.223104,47.810825,3.811378,9


In [54]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('Dreal', 'Dfake', 'Dfull', 'G_adv', 'G_f…

<function __main__.f_plot_metrics(df, col_list)>

In [55]:

chi=df_metrics.quantile(q=0.005,axis=0)['hist_chi']
print(chi)
df_metrics[df_metrics['hist_chi']<=chi].sort_values(by=['hist_chi']).head(10)

2.312528178691864


,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,step
30,150.263596,5074.983398,5225.247070,0.000000,51.945053,51.945053,-4.326087,52.301987,-6.853485,30
6,172.202179,8064.131836,8236.333984,0.000000,48.534889,48.534889,2.687699,49.605465,-5.016946,6
56,122.571365,3536.480957,3659.052246,0.000000,48.903160,48.903160,2.584758,50.406216,0.569272,56
24886,0.398336,0.321099,0.719435,32.800297,36.447552,36.447552,2.563627,44.500336,1.889011,24886
18102,0.877707,0.624670,1.502376,37.664513,35.243202,35.243202,2.686904,42.105152,1.988223,18102
17282,1.169223,1.304721,2.473945,84.383606,36.875671,36.875671,2.643668,42.296188,2.025215,17282
19089,0.409318,0.685260,1.094579,18.610264,34.804489,34.804489,2.628868,43.030487,2.031394,19089
14866,2.524513,1.167857,3.692370,39.873764,36.280365,36.280365,2.749026,42.411461,2.047975,14866
15466,0.872521,1.143690,2.016211,52.832001,38.074795,38.074795,2.728985,41.897797,2.056871,15466
17303,0.860687,0.612616,1.473303,29.519232,36.646690,36.646690,2.600482,41.438969,2.069997,17303


In [64]:
display(df_metrics.sort_values(by=['hist_chi']).head(8))
display(df_metrics.sort_values(by=['spec_loss']).head(8))

,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,step
30,150.263596,5074.983398,5225.247070,0.000000,51.945053,51.945053,-4.326087,52.301987,-6.853485,30
6,172.202179,8064.131836,8236.333984,0.000000,48.534889,48.534889,2.687699,49.605465,-5.016946,6
56,122.571365,3536.480957,3659.052246,0.000000,48.903160,48.903160,2.584758,50.406216,0.569272,56
24886,0.398336,0.321099,0.719435,32.800297,36.447552,36.447552,2.563627,44.500336,1.889011,24886
18102,0.877707,0.624670,1.502376,37.664513,35.243202,35.243202,2.686904,42.105152,1.988223,18102
17282,1.169223,1.304721,2.473945,84.383606,36.875671,36.875671,2.643668,42.296188,2.025215,17282
19089,0.409318,0.685260,1.094579,18.610264,34.804489,34.804489,2.628868,43.030487,2.031394,19089
14866,2.524513,1.167857,3.692370,39.873764,36.280365,36.280365,2.749026,42.411461,2.047975,14866


,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,step
27700,0.595195,0.144487,0.739682,9.269048,28.939371,28.939371,2.543119,36.922581,2.738028,27700
29640,0.450233,0.164217,0.614451,6.213902,28.967270,28.967270,2.473536,35.569794,2.706482,29640
25082,0.882019,0.707787,1.589806,67.775887,29.640848,29.640848,2.635364,37.535686,2.810772,25082
30256,0.115976,0.426527,0.542503,27.548754,29.678890,29.678890,2.604787,37.064125,2.607141,30256
28567,0.759417,0.482429,1.241846,23.956360,29.691013,29.691013,2.567212,35.945518,2.589561,28567
26347,0.961231,0.449026,1.410257,27.191336,29.693794,29.693794,2.684590,37.799088,2.679707,26347
23898,0.497465,0.271034,0.768500,11.523281,29.788918,29.788918,2.686743,36.070263,2.634746,23898
23817,0.440706,0.150620,0.591326,17.327530,29.799273,29.799273,2.675605,36.811886,2.700470,23817


## Plots for selected images

In [57]:
# Read input data

fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
samples=np.load(fname,mmap_mode='r+')[:2000]
samples=samples[:,0,:,:]
print(samples.shape)

(2000, 128, 128)


In [65]:
### Read GAN results
# epoch,step=0,100
# fname='gen_img_epoch-{0}_step-{1}.npy'.format(epoch,step)

step=27700

fname='*gen_img_epoch-*_step-{0}.npy'.format(step)
# fname='gen_img_epoch-*_step-{0}.npy'.format(step)
fle=glob.glob(result_dir+'/images/'+fname)[0]
fle
imgs=np.load(fle)
print(imgs.shape)

(64, 128, 128)


In [66]:
# keras image for comparison
f1='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/exagan1/run5_fixed_cosmology/models/gen_imgs.npy'
img_2=np.expand_dims(np.load(f1)[:500],axis=1)[:,0,:,:]
print(img_2.shape)

(500, 128, 128)


In [67]:
# imgs=f_invtransform(imgs)
# samples=f_invtransform(samples)

### Plot images

In [68]:
f_plot_grid(samples[100:118],cols=6,fig_size=(6,3))
f_plot_grid(imgs[10:28],cols=6,fig_size=(6,3))

3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Plot Histogram

In [69]:

img_lst=[f_invtransform(imgs)]
# img_lst=[f_invtransform(img_2)]

# img_lst=[samples]
# f_pixel_intensity(f_invtransform(imgs),bins=25,label='validation',mode='avg',normalize=False,log_scale=True,plot=True, hist_range=(0,1200))
# f_pixel_intensity(f_invtransform(img_2),bins=25,label='validation',mode='avg',normalize=False,log_scale=True,plot=True, hist_range=None)

f_compare_pixel_intensity(img_lst,label_lst=['imgs'],bkgnd_arr=f_invtransform(samples),log_scale=True, normalize=True, mode='avg',bins=50, hist_range=(0,1200))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Plot spectrum

In [71]:
f_compare_spectrum(img_lst,label_lst=['img'],bkgnd_arr=f_invtransform(samples),log_scale=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Read stored chi-squares for images

In [34]:
df=pd.read_pickle(result_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)

chi1=df.quantile(q=0.2,axis=0)['chi_1']
chi_spec=df.quantile(q=0.2,axis=0)['chi_spec1']

df[(df.chi_spec1<chi_spec)&(df.chi_1<chi1)].sort_values(by=['chi_spec1','chi_1']).head(10)
# df[df.chi_1<chi1].sort_values(by=['chi_1']).head(5)


,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_err
483,7,24100,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.055965,0.049232,0.005330,0.110526,0.055218,8.832181,0.722093,5762.308428,0,0,"[1.2482491167794125, 0.7657722842568389, 0.251...","[0.006473762797512475, 0.0065813500594378545, ...","[-1.031746031584782, -0.6161616169043975, -0.3...","[74987.0960796997, 58784.30722787454, 47123.52...","[5023.286887524211, 2559.5794595366688, 1962.1..."
527,8,26300,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.064215,0.036092,0.009525,0.109832,0.061145,10.037659,0.776200,2856.110049,0,0,"[1.2631737324394756, 0.7627392422981261, 0.232...","[0.009216444976011717, 0.007161635158132146, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[73739.69629648328, 51357.23982004525, 40004.4...","[4136.030149777228, 2177.8602734946144, 1512.9..."
387,6,19300,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.029178,0.015428,0.035088,0.079694,0.029075,10.453338,0.784865,2931.900345,2,0,"[1.2110436589960687, 0.817649693556493, 0.2538...","[0.008151849119926155, 0.006037037980511502, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[69772.1652399078, 55105.37390130758, 46112.51...","[4073.3417775460202, 2119.5619477168702, 1505...."
524,8,26150,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.014157,0.009960,0.021237,0.045354,0.013021,13.671631,0.785724,1799.561962,0,0,"[1.1725539595330807, 0.8612216426287383, 0.271...","[0.012894638705463946, 0.009704499196507956, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[84066.41643619537, 61105.092987811586, 49132....","[4962.041118605277, 2522.752879795386, 2101.70..."
523,8,26100,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.096103,0.009013,0.004690,0.109805,0.094147,11.350340,0.825873,3775.456402,0,0,"[1.2923170063450609, 0.7087640773246933, 0.234...","[0.010113655778567479, 0.006728217216069455, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[68989.31879784167, 50714.86580108999, 43833.1...","[5474.964132709261, 3116.810844642119, 2177.14..."
582,9,29050,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.033194,0.035245,0.004243,0.072681,0.031349,12.408207,0.865288,1833.654969,0,0,"[1.2146729560234808, 0.8132063460955775, 0.255...","[0.012891273452769457, 0.009829231073548864, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[96575.29732233286, 63044.92819120883, 47429.9...","[5216.357602901766, 2692.6329119119846, 2267.8..."
515,8,25700,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.067889,0.009948,0.008130,0.085967,0.063728,8.643314,0.914018,5058.414229,0,0,"[1.2507518860176643, 0.7457211625142209, 0.255...","[0.006421856821490693, 0.006417874110540535, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[92935.39532610029, 57708.33834919632, 44722.4...","[2570.64162523229, 1441.9948848433623, 1437.98..."
450,7,22450,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.045305,0.004842,0.056623,0.106769,0.043371,11.554129,1.060685,5805.835665,0,0,"[1.2280730040755459, 0.7834719133524779, 0.254...","[0.008320320545552227, 0.006714473091392117, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[75326.88581323996, 63329.87507540136, 48770.5...","[8606.647896026652, 4285.48321571592, 2208.931..."
458,7,22850,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.020820,0.002872,0.005872,0.029563,0.009816,9.535744,1.061469,5268.106154,0,0,"[1.144295009941987, 0.8751240684161449, 0.2906...","[0.006876944877376865, 0.00447251114802238, 0....","[-1.031746031584782, -0.6161616169043975, -0.3...","[90509.3457731232, 58173.688622455295, 43899.2...","[3862.0878085683503, 1717.2836728214577, 1486...."
555,8,27700,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.083898,0.003867,0.007060,0.094826,0.073218,10.468445,1.065720,3432.722027,1,0,"[1.28173327776589

In [35]:
df[(df.chi_spec1<chi_spec)&(df.chi_1<chi1)].sort_values(by=['chi_1','chi_spec1']).head(5)
# df[df.chi_spec1<chi_spec].sort_values(by=['chi_spec1']).head(5)


,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_err
458,7,22850,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.020820,0.002872,0.005872,0.029563,0.009816,9.535744,1.061469,5268.106154,0,0,"[1.144295009941987, 0.8751240684161449, 0.2906...","[0.006876944877376865, 0.00447251114802238, 0....","[-1.031746031584782, -0.6161616169043975, -0.3...","[90509.3457731232, 58173.688622455295, 43899.2...","[3862.0878085683503, 1717.2836728214577, 1486...."
412,6,20550,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.022883,0.007903,0.004400,0.035186,0.018544,11.891742,1.576272,2852.810103,0,0,"[1.1890801222000666, 0.8449942780636412, 0.252...","[0.01039005253948027, 0.0077831216492957045, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[90745.257419914, 74035.91218162775, 52412.395...","[3248.636676879511, 2420.257818615122, 2337.47..."
509,8,25400,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.013866,0.017516,0.009815,0.041198,0.003972,10.317339,1.091298,3209.933342,0,0,"[1.1304800241542472, 0.9085907950092165, 0.278...","[0.008974678503265115, 0.006758624208107186, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[75118.69395630062, 61021.649852347386, 48254....","[3668.928391656569, 2089.0948622383385, 1754.1..."
553,8,27600,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.018017,0.016315,0.007040,0.041372,0.013369,9.819966,1.422457,6050.820209,0,0,"[1.168265643749009, 0.8586636198872947, 0.2737...","[0.007325125240150375, 0.004779489242961735, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[91952.573633641, 51282.23242847025, 41585.641...","[3094.9193334028287, 2357.1905626135785, 1841...."
595,9,29700,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.004629,0.032600,0.004466,0.041695,0.002345,8.332750,1.214480,6157.903359,0,0,"[1.1298623114165247, 0.9232190265571707, 0.278...","[0.0059527397917583475, 0.004291547065615404, ...","[-1.031746031584782, -0.6161616169043975, -0.3...","[66943.23146651685, 55974.42372954935, 49484.9...","[3403.004470491841, 1495.325472432646, 1529.58..."


In [37]:
step_list=[24100,26300,22850,20550]
df_metrics[df_metrics.step.isin(step_list)]


,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,step
20550,0.933440,3.920427,4.853867,99.513840,99.513840,48.716425,-4.487527,47.573264,-10.508814,20550
22850,1.617100,0.444844,2.061944,60.804123,60.804123,46.702021,-5.102414,46.102859,-5.098503,22850
24100,2.096158,0.607756,2.703914,80.565552,80.565552,47.943841,-5.226744,47.575316,-5.386455,24100
26300,6.530154,0.536849,7.067003,73.086838,73.086838,47.652099,-4.809917,48.056364,-7.812605,26300


In [ ]:
df[df.step.isin(step_list)][['epoch','step','chi_1','chi_spec1']]


In [ ]:
df_metrics.sort_values(by=['hist_chi']).head(8)